In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, f1_score


In [ ]:
df=pd.read_csv("C:\\Users\\dipan\\Downloads\\Churn-Data.csv")
df=pd.get_dummies(df)
df.fillna(df.median(), inplace=True)
x=df.drop('Churn',axis=1)
y=df['Churn']
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
smote=SMOTE(random_sample=42)
x_res,y_res=smote.fit_resample(x_scaled,y)
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42)
scoring = {'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score)}
#random forest classifier model
rf=RandomForestClassifier(random_sample=42)
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
rf_grid = GridSearchCV(rf, rf_params, scoring=scoring, refit='f1', cv=5)
rf_grid.fit(x_train, y_train)
rf_best = rf_grid.best_estimator
print("Random Forest Best Params:", rf_grid.best_params_)
# Evaluating on the test set
rf_pred = rf_best.predict(x_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)
print(f"Random Forest - Accuracy: {rf_accuracy}, F1-score: {rf_f1}")

In [ ]:
#Support vector classifier model
svc = SVC(random_state=42)
svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}
svc_grid = GridSearchCV(svc, svc_params, scoring=scoring, refit='f1', cv=5)
svc_grid.fit(x_train,y_train)
# Results of the grid search
svc_best = svc_grid.best_estimator_
print("SVC Best Params:", svc_grid.best_params_)
# Evaluating on the test set
svc_pred = svc_best.predict(x_test)
svc_accuracy = accuracy_score(y_test, svc_pred)
svc_f1 = f1_score(y_test, svc_pred)
print(f"SVC - Accuracy: {svc_accuracy}, F1-score: {svc_f1}")